In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests

In [2]:
# --- 1. CREDENCIALES  ---

CLIENT_ID = 'c28947c71dcd47e1b8a406ded3bd29fc'
CLIENT_SECRET = 'b9e1640a795446be91d8b87d1cca7bda'



In [3]:
# Autenticación
try:
    auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    print("✅ Autenticación con Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de autenticación: {e}")
    exit()

✅ Autenticación con Spotify exitosa.


In [4]:
datos = sp.search(q="genre:indie year:2010", type='track', limit=50, offset=0) #Obtenemos los 50 primeros

In [5]:
datos

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=50&query=genre%3Aindie%20year%3A2010&type=track',
  'limit': 50,
  'next': 'https://api.spotify.com/v1/search?offset=50&limit=50&query=genre%3Aindie%20year%3A2010&type=track',
  'offset': 0,
  'previous': None,
  'total': 100,
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6lcwlkAjBPSKnFBZjjZFJs'},
       'href': 'https://api.spotify.com/v1/artists/6lcwlkAjBPSKnFBZjjZFJs',
       'id': '6lcwlkAjBPSKnFBZjjZFJs',
       'name': 'Alex G',
       'type': 'artist',
       'uri': 'spotify:artist:6lcwlkAjBPSKnFBZjjZFJs'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'H

In [6]:
datos['tracks']['items'][0] # Inspeccionamos el primer track a ver los datos que nos da y cuales podemos obtener

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6lcwlkAjBPSKnFBZjjZFJs'},
    'href': 'https://api.spotify.com/v1/artists/6lcwlkAjBPSKnFBZjjZFJs',
    'id': '6lcwlkAjBPSKnFBZjjZFJs',
    'name': 'Alex G',
    'type': 'artist',
    'uri': 'spotify:artist:6lcwlkAjBPSKnFBZjjZFJs'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
   'K

In [7]:
GENRE_TARGET = 'indie' 
YEARS_LIST = [2010, 2015, 2020, 2025] 
SPOTIFY_MAX_LIMIT = 1000

results = []
artistas = []

In [8]:


print("--- Inicio de la Extracción Masiva de Spotify ---")

# --- BUCLE PRINCIPAL DE EXTRACCIÓN: Iterar por año ---

for year in YEARS_LIST:
    print(f"\n>>>> Extrayendo datos para el género '{GENRE_TARGET}' en el año: {year}")
    
    offset = 0 # Reinicia el desplazamiento para cada año
    
    while offset < SPOTIFY_MAX_LIMIT:
        
        # 1. Definición de la consulta (género y año específicos)
        query = f'genre:{GENRE_TARGET} year:{year}'
        
        # 2. Llamada a la API de Spotify (SIN PAUSA DE TIEMPO)
        try:
            response = sp.search(q=query, limit=50, offset=offset, type='track,album')
        except Exception as e:
            print(f"❌ Error en la API de Spotify en offset {offset}: {e}")
            break
        
        # --- PROCESAMIENTO DE TRACKS (Canciones) ---
        
        current_tracks = response['tracks']['items']
        if not current_tracks:
            break

        for item in current_tracks:
            artist_name = item['artists'][0]['name'] if item['artists'] else 'Artista Desconocido'
            artistas.append(artist_name)
            
            results.append({
                'name_track': item['name'],
                'name_artist': artist_name,
                'genre': GENRE_TARGET, 
                'type': 'track', 
                'year': str(year),
            })

        # --- PROCESAMIENTO DE ALBUMS (Solo en la primera página) ---
        
        if offset == 0 and 'albums' in response:
            for item in response['albums']['items']:
                artist_name = item['artists'][0]['name'] if item['artists'] else 'Artista Desconocido'
                artistas.append(artist_name)
                
                results.append({
                    'name_track': item['name'],
                    'name_artist': artist_name,
                    'genre': GENRE_TARGET, 
                    'type': 'album', 
                    'year': str(year),
                })

        # --- AVANCE DE PÁGINA ---
        offset += 50 
        
        if offset % 100 == 0:
            print(f"   -> Offset actual: {offset}. Registros totales recolectados: {len(results)}")


# --- FIN DEL PROCESO ---
artistas_unicos = list(set(artistas)) 
print("\n--- Resumen de Extracción ---")
print(f"CONTEO FINAL: Total de registros (tracks + albums) recolectados: {len(results)}")
print(f"Artistas Únicos recolectados para la fase Last.fm: {len(artistas_unicos)}")

--- Inicio de la Extracción Masiva de Spotify ---

>>>> Extrayendo datos para el género 'indie' en el año: 2010
   -> Offset actual: 100. Registros totales recolectados: 100
   -> Offset actual: 200. Registros totales recolectados: 200
   -> Offset actual: 300. Registros totales recolectados: 300
   -> Offset actual: 400. Registros totales recolectados: 400
   -> Offset actual: 500. Registros totales recolectados: 500
   -> Offset actual: 600. Registros totales recolectados: 600
   -> Offset actual: 700. Registros totales recolectados: 700
   -> Offset actual: 800. Registros totales recolectados: 800
   -> Offset actual: 900. Registros totales recolectados: 900
   -> Offset actual: 1000. Registros totales recolectados: 1000

>>>> Extrayendo datos para el género 'indie' en el año: 2015
   -> Offset actual: 100. Registros totales recolectados: 1100
   -> Offset actual: 200. Registros totales recolectados: 1200
   -> Offset actual: 300. Registros totales recolectados: 1300
   -> Offset ac

In [9]:
# --- CONFIGURACIÓN DE ARCHIVOS ---
SPOTIFY_OUTPUT_FILE = 'spotify_base_datos_final.csv'
ARTISTS_OUTPUT_FILE = 'lastfm_unique_artists.csv'

# --- 1. PROCESAR DATOS PRINCIPALES (Tracks y Albums) ---
print("\n--- Sección 3: Guardado de Datos ---")
# La variable 'results' viene de la Sección 2 ejecutada anteriormente.
print(f"Total de registros a guardar: {len(results)}") 

df_final = pd.DataFrame(results)

# Seleccionar y renombrar las columnas para cumplir con el enunciado
df_final_renombrado = df_final.rename(columns={
    'name_artist': 'Artista',
    'genre': 'Genero_Musical_Seed',
    'type': 'Cancion_o_Album_Tipo',
    'name_track': 'Nombre_Cancion_o_Album',
    'year': 'Año_Lanzamiento'
})

# Eliminamos duplicados
initial_count = len(df_final_renombrado)
df_final_renombrado.drop_duplicates(subset=['Artista', 'Nombre_Cancion_o_Album', 'Año_Lanzamiento'], inplace=True)
print(f"-> Se eliminaron {initial_count - len(df_final_renombrado)} duplicados. Registros únicos: {len(df_final_renombrado)}")

# Guardar la base de datos principal de Spotify
df_final_renombrado.to_csv(SPOTIFY_OUTPUT_FILE, index=False)
print(f"✅ Base de Datos de Spotify guardada como: '{SPOTIFY_OUTPUT_FILE}'")

# --- 2. PROCESAR ARTISTAS ÚNICOS PARA LAST.FM ---

# Crear el DataFrame de artistas únicos
# La variable 'artistas' viene de la Sección 2.
df_artists = pd.DataFrame(list(set(artistas)), columns=['artist_name'])
print(f"Total de artistas únicos para Last.fm: {len(df_artists)}")

# Guardar la lista de artistas únicos
df_artists.to_csv(ARTISTS_OUTPUT_FILE, index=False)
print(f"✅ Lista de artistas para Last.fm guardada como: '{ARTISTS_OUTPUT_FILE}'")

# --- Muestra de la Base de Datos Final (CORRECCIÓN APLICADA AQUÍ) ---
columnas_muestra = ['Artista', 'Genero_Musical_Seed', 'Cancion_o_Album_Tipo', 'Nombre_Cancion_o_Album', 'Año_Lanzamiento']
print("\n--- Muestra de la Base de Datos Final (Primeras 5 Filas) ---")
# Usamos .head() en lugar de .to_markdown() para evitar el error de 'tabulate'
print(df_final_renombrado[columnas_muestra].head(5)) 

print("\n--- ¡Spotify y Guardado en CSVs Completados! ---")


--- Sección 3: Guardado de Datos ---
Total de registros a guardar: 4006
-> Se eliminaron 248 duplicados. Registros únicos: 3758
✅ Base de Datos de Spotify guardada como: 'spotify_base_datos_final.csv'
Total de artistas únicos para Last.fm: 1598
✅ Lista de artistas para Last.fm guardada como: 'lastfm_unique_artists.csv'

--- Muestra de la Base de Datos Final (Primeras 5 Filas) ---
         Artista Genero_Musical_Seed Cancion_o_Album_Tipo  \
0         Alex G               indie                track   
1  Miss Caffeina               indie                track   
2   Pedro Guerra               indie                track   
3     Phantogram               indie                track   
4           IZAL               indie                track   

    Nombre_Cancion_o_Album Año_Lanzamiento  
0                     Gnaw            2010  
1      Mi rutina preferida            2010  
2  Contigo en la Distancia            2010  
3     Mouthful of Diamonds            2010  
4                 Desord